In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the tokenizer and model for FinBERT
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Create the sentiment-analysis pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# Example sentences related to Indian stock market news
sentences = [
    "The Indian stock market experienced a surge in trading activity."
]

# Perform sentiment analysis using the FinBERT model for all labels
results = pipe(sentences)

# Print the results
for result in results:
    for label_score in result:
        print(f"Label: {label_score['label']}, Score: {label_score['score']}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Label: positive, Score: 0.942677915096283
Label: negative, Score: 0.0227152518928051
Label: neutral, Score: 0.03460685908794403


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datetime import datetime

# Load FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Create the NLP pipeline
nlp_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)

# List of stock symbols
stock_symbols = [
     "SBIN"
]

# Path to the news files
data_path = "/content/drive/MyDrive/Dataset/news/"

def calculate_sentiment_for_stocks():
    for stock in stock_symbols:
        # Load the stock's news data
        file_path = f"{data_path}{stock}.csv"
        df = pd.read_csv(file_path)

        # Ensure that the date is in datetime format
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

        # Remove rows with dates before 2021
        df = df[df["date"].dt.year >= 2021]

        # Initialize columns for results
        df["Pt_plus"] = np.nan
        df["Pt_minus"] = np.nan
        df["Dt"] = np.nan

        # Process each row
        for index, row in df.iterrows():
            date = row["date"]
            description = row.get("description", "")

            # Ensure description is a valid string
            if isinstance(description, str) and description.strip():
                try:
                    # Get sentiment scores from FinBERT, with truncation
                    results = nlp_pipeline(description)

                    # Verify and parse the sentiment output structure
                    if results and isinstance(results[0], list):
                        # Extract positive and negative scores
                        positive_score = next((item["score"] for item in results[0] if item["label"] == "positive"), 0.0)
                        negative_score = next((item["score"] for item in results[0] if item["label"] == "negative"), 0.0)

                        # Calculate Pt_plus, Pt_minus, and Dt
                        total = positive_score + negative_score
                        if total > 0:
                            Pt_plus = positive_score / total
                            Pt_minus = negative_score / total
                            Dt = Pt_plus - Pt_minus
                        else:
                            Pt_plus = Pt_minus = Dt = 0.0

                        # Store the results in the DataFrame
                        df.at[index, "Pt_plus"] = Pt_plus
                        df.at[index, "Pt_minus"] = Pt_minus
                        df.at[index, "Dt"] = Dt
                    else:
                        print(f"Unexpected result format at index {index}: {results}")
                except Exception as e:
                    # Skip rows where there are errors
                    print(f"Error processing row {index}: {e}")
                    df.at[index, "Pt_plus"] = df.at[index, "Pt_minus"] = df.at[index, "Dt"] = np.nan
            else:
                # Handle rows where the description is not valid (NaN or empty)
                df.at[index, "Pt_plus"] = df.at[index, "Pt_minus"] = df.at[index, "Dt"] = np.nan

        # Group by date and calculate the average Pt_plus, Pt_minus, and Dt for each day
        daily_sentiment = df.groupby(df["date"].dt.date).agg(
            Pt_plus=("Pt_plus", "mean"),
            Pt_minus=("Pt_minus", "mean"),
            Dt=("Dt", "mean")
        ).reset_index()

        # Save the updated DataFrame with the daily averages to a new CSV file
        output_file = f"{data_path}{stock}_sentiment.csv"
        daily_sentiment.to_csv(output_file, index=False)
        print(f"Sentiment data for {stock} saved to {output_file}")

# Run the function
calculate_sentiment_for_stocks()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (589 > 512). Running this sequence through the model will result in indexing errors


Error processing row 7563: The size of tensor a (589) must match the size of tensor b (512) at non-singleton dimension 1
Sentiment data for SBIN saved to /content/drive/MyDrive/Dataset/news/SBIN_sentiment.csv


In [ ]:
output

{'error': 'Rate limit reached. You reached free usage limit (reset daily). Please subscribe to PRO or Enterprise Hub to get a higher limit: https://hf.co/pricing'}